In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/KY_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1495627205,KY,"424 LEWIS HARGETT CIR Ste 250, LEXINGTON, KY",37.995388,-84.534780,2.106700e+14
1,2484477207,KY,"1125 MADISON AVENUE, COVINGTON, KY",39.078137,-84.508961,2.111707e+14
2,5057388504,KY,"295 N Hubbards Ln Ste 302, Louisville, KY",38.259761,-85.639661,2.111101e+14
3,5380597000,KY,"2501 Crossing Blvd. Suite 300, BOWLING GREEN, KY",36.941655,-86.434784,2.122701e+14
4,5452197107,KY,"101 BULLITT LN Suite 205, LOUISVILLE, KY",38.245828,-85.616841,2.111101e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1495627205,KY,"424 LEWIS HARGETT CIR Ste 250, LEXINGTON, KY",37.995388,-84.534780,2.106700e+14
1,2484477207,KY,"1125 MADISON AVENUE, COVINGTON, KY",39.078137,-84.508961,2.111707e+14
2,5057388504,KY,"295 N Hubbards Ln Ste 302, Louisville, KY",38.259761,-85.639661,2.111101e+14
3,5380597000,KY,"2501 Crossing Blvd. Suite 300, BOWLING GREEN, KY",36.941655,-86.434784,2.122701e+14
4,5452197107,KY,"101 BULLITT LN Suite 205, LOUISVILLE, KY",38.245828,-85.616841,2.111101e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1115,7982897003,KY,"MARKET ST, LOUISVILLE, KY",38.270696,-85.739525,NaN
1537,8865637404,KY,"1003 HIGHWAY 11, MANCHESTER, KY",38.132460,-79.062841,NaN
1855,6546147000,KY,"170 Commerce Road, CONWAY, KY",35.071898,-92.430098,NaN
2831,7204268906,KY,"8616 La Tijera Blvd., Milton, KY",33.959230,-118.391504,NaN
2961,6546067000,KY,"8363 VETERANS MEMORIAL HWY, SHARPSBURG, KY",33.732568,-84.844896,NaN
3165,8292928504,KY,"35 DELANEY PL, NICHOLASVILLE, KY",37.973367,-84.617537,NaN
3454,3114768302,KY,"1505 S 77th Street, Louisville, KY",43.014747,-88.008735,NaN
3657,6918028400,KY,"2987 VETERANS MEMORIAL HWY, SHARPSBURG, KY",40.788891,-73.145920,NaN
3972,9814137009,KY,"2987 Veterans Memorial Highway, SHARPSBURG, KY",40.788891,-73.145920,NaN
4241,8166797102,KY,"5 US HWY 68 WEST, BENTON, KY",40.819345,-83.651431,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('21')] #KY
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/17 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

17


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_12955/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1115,7982897003,KY,"MARKET ST, LOUISVILLE, KY",38.270696,-85.739525,[]
1537,8865637404,KY,"1003 HIGHWAY 11, MANCHESTER, KY",38.132460,-79.062841,[]
1855,6546147000,KY,"170 Commerce Road, CONWAY, KY",35.071898,-92.430098,[]
2831,7204268906,KY,"8616 La Tijera Blvd., Milton, KY",33.959230,-118.391504,[]
2961,6546067000,KY,"8363 VETERANS MEMORIAL HWY, SHARPSBURG, KY",33.732568,-84.844896,[]
3165,8292928504,KY,"35 DELANEY PL, NICHOLASVILLE, KY",37.973367,-84.617537,[211130606003014]
3454,3114768302,KY,"1505 S 77th Street, Louisville, KY",43.014747,-88.008735,[]
3657,6918028400,KY,"2987 VETERANS MEMORIAL HWY, SHARPSBURG, KY",40.788891,-73.145920,[]
3972,9814137009,KY,"2987 Veterans Memorial Highway, SHARPSBURG, KY",40.788891,-73.145920,[]
4241,8166797102,KY,"5 US HWY 68 WEST, BENTON, KY",40.819345,-83.651431,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_12955/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1115,7982897003,KY,"MARKET ST, LOUISVILLE, KY",38.270696,-85.739525,NaN
1537,8865637404,KY,"1003 HIGHWAY 11, MANCHESTER, KY",38.132460,-79.062841,NaN
1855,6546147000,KY,"170 Commerce Road, CONWAY, KY",35.071898,-92.430098,NaN
2831,7204268906,KY,"8616 La Tijera Blvd., Milton, KY",33.959230,-118.391504,NaN
2961,6546067000,KY,"8363 VETERANS MEMORIAL HWY, SHARPSBURG, KY",33.732568,-84.844896,NaN
3165,8292928504,KY,"35 DELANEY PL, NICHOLASVILLE, KY",37.973367,-84.617537,211130606003014
3454,3114768302,KY,"1505 S 77th Street, Louisville, KY",43.014747,-88.008735,NaN
3657,6918028400,KY,"2987 VETERANS MEMORIAL HWY, SHARPSBURG, KY",40.788891,-73.145920,NaN
3972,9814137009,KY,"2987 Veterans Memorial Highway, SHARPSBURG, KY",40.788891,-73.145920,NaN
4241,8166797102,KY,"5 US HWY 68 WEST, BENTON, KY",40.819345,-83.651431,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1115,7982897003,KY,"MARKET ST, LOUISVILLE, KY",38.270696,-85.739525,NaN
1537,8865637404,KY,"1003 HIGHWAY 11, MANCHESTER, KY",38.132460,-79.062841,NaN
1855,6546147000,KY,"170 Commerce Road, CONWAY, KY",35.071898,-92.430098,NaN
2831,7204268906,KY,"8616 La Tijera Blvd., Milton, KY",33.959230,-118.391504,NaN
2961,6546067000,KY,"8363 VETERANS MEMORIAL HWY, SHARPSBURG, KY",33.732568,-84.844896,NaN
3454,3114768302,KY,"1505 S 77th Street, Louisville, KY",43.014747,-88.008735,NaN
3657,6918028400,KY,"2987 VETERANS MEMORIAL HWY, SHARPSBURG, KY",40.788891,-73.145920,NaN
3972,9814137009,KY,"2987 Veterans Memorial Highway, SHARPSBURG, KY",40.788891,-73.145920,NaN
4241,8166797102,KY,"5 US HWY 68 WEST, BENTON, KY",40.819345,-83.651431,NaN
4373,7295787101,KY,"8415 BELIZE PLACE WEST PALM BEACH, FL 33414, L...",26.644779,-80.180423,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
1115,7982897003,NaN
1537,8865637404,NaN
1855,6546147000,NaN
2831,7204268906,NaN
2961,6546067000,NaN
3165,8292928504,211130606003014
3454,3114768302,NaN
3657,6918028400,NaN
3972,9814137009,NaN
4241,8166797102,NaN


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/KY_fips_scraped.csv") 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,373697,1495627205,KY,"424 LEWIS HARGETT CIR Ste 250, LEXINGTON, KY",37.995388,-84.534780,2.106700e+14
1,373698,2484477207,KY,"1125 MADISON AVENUE, COVINGTON, KY",39.078137,-84.508961,2.111707e+14
2,373699,5057388504,KY,"295 N Hubbards Ln Ste 302, Louisville, KY",38.259761,-85.639661,2.111101e+14
3,373700,5380597000,KY,"2501 Crossing Blvd. Suite 300, BOWLING GREEN, KY",36.941655,-86.434784,2.122701e+14
4,373701,5452197107,KY,"101 BULLITT LN Suite 205, LOUISVILLE, KY",38.245828,-85.616841,2.111101e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,373697,1495627205,KY,"424 LEWIS HARGETT CIR Ste 250, LEXINGTON, KY",37.995388,-84.534780,2.106700e+14,NaN
1,373698,2484477207,KY,"1125 MADISON AVENUE, COVINGTON, KY",39.078137,-84.508961,2.111707e+14,NaN
2,373699,5057388504,KY,"295 N Hubbards Ln Ste 302, Louisville, KY",38.259761,-85.639661,2.111101e+14,NaN
3,373700,5380597000,KY,"2501 Crossing Blvd. Suite 300, BOWLING GREEN, KY",36.941655,-86.434784,2.122701e+14,NaN
4,373701,5452197107,KY,"101 BULLITT LN Suite 205, LOUISVILLE, KY",38.245828,-85.616841,2.111101e+14,NaN
...,...,...,...,...,...,...,...,...
9173,382870,9583298303,KY,"1494 Ivan Downs Blvd, Bowling Green, KY",36.918635,-86.469539,2.122701e+14,NaN
9174,382871,9591048407,KY,"169 E Reynolds Rd, Lexington, KY",37.990967,-84.519720,2.106700e+14,NaN
9175,382872,9642117104,KY,"10 HAMPTON LN, WILDER, KY",39.025711,-84.479734,2.103705e+14,NaN
9176,382873,9721428609,KY,"129 Walton Ave, Lexington, KY",38.037605,-84.484954,2.106700e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,373697,1495627205,KY,"424 LEWIS HARGETT CIR Ste 250, LEXINGTON, KY",37.995388,-84.534780,2.106700e+14,NaN,210670042041002.0
1,373698,2484477207,KY,"1125 MADISON AVENUE, COVINGTON, KY",39.078137,-84.508961,2.111707e+14,NaN,211170671003001.0
2,373699,5057388504,KY,"295 N Hubbards Ln Ste 302, Louisville, KY",38.259761,-85.639661,2.111101e+14,NaN,211110100013007.0
3,373700,5380597000,KY,"2501 Crossing Blvd. Suite 300, BOWLING GREEN, KY",36.941655,-86.434784,2.122701e+14,NaN,212270107022001.0
4,373701,5452197107,KY,"101 BULLITT LN Suite 205, LOUISVILLE, KY",38.245828,-85.616841,2.111101e+14,NaN,211110107014001.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1495627205,KY,"424 LEWIS HARGETT CIR Ste 250, LEXINGTON, KY",37.995388,-84.534780,210670042041002.0
1,2484477207,KY,"1125 MADISON AVENUE, COVINGTON, KY",39.078137,-84.508961,211170671003001.0
2,5057388504,KY,"295 N Hubbards Ln Ste 302, Louisville, KY",38.259761,-85.639661,211110100013007.0
3,5380597000,KY,"2501 Crossing Blvd. Suite 300, BOWLING GREEN, KY",36.941655,-86.434784,212270107022001.0
4,5452197107,KY,"101 BULLITT LN Suite 205, LOUISVILLE, KY",38.245828,-85.616841,211110107014001.0
...,...,...,...,...,...,...
9173,9583298303,KY,"1494 Ivan Downs Blvd, Bowling Green, KY",36.918635,-86.469539,212270108021033.0
9174,9591048407,KY,"169 E Reynolds Rd, Lexington, KY",37.990967,-84.519720,210670035011003.0
9175,9642117104,KY,"10 HAMPTON LN, WILDER, KY",39.025711,-84.479734,210370533022010.0
9176,9721428609,KY,"129 Walton Ave, Lexington, KY",38.037605,-84.484954,210670005001011.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1115,7982897003,KY,"MARKET ST, LOUISVILLE, KY",38.270696,-85.739525,NaN
1537,8865637404,KY,"1003 HIGHWAY 11, MANCHESTER, KY",38.132460,-79.062841,NaN
1855,6546147000,KY,"170 Commerce Road, CONWAY, KY",35.071898,-92.430098,NaN
2831,7204268906,KY,"8616 La Tijera Blvd., Milton, KY",33.959230,-118.391504,NaN
2961,6546067000,KY,"8363 VETERANS MEMORIAL HWY, SHARPSBURG, KY",33.732568,-84.844896,NaN
3454,3114768302,KY,"1505 S 77th Street, Louisville, KY",43.014747,-88.008735,NaN
3657,6918028400,KY,"2987 VETERANS MEMORIAL HWY, SHARPSBURG, KY",40.788891,-73.145920,NaN
3972,9814137009,KY,"2987 Veterans Memorial Highway, SHARPSBURG, KY",40.788891,-73.145920,NaN
4241,8166797102,KY,"5 US HWY 68 WEST, BENTON, KY",40.819345,-83.651431,NaN
4373,7295787101,KY,"8415 BELIZE PLACE WEST PALM BEACH, FL 33414, L...",26.644779,-80.180423,NaN


Convert to File

In [19]:
fips_KY = fips_merge_drop

In [20]:
fips_KY.to_csv("../../../data/state_data/geo/geo_fips/23/KY_fips_scraped.csv") 